###  스팸필터
    -홍길동 받은 메일의 80% 스팸 메일  
    -스팸 메일의 95%에서 '대출'이라는 단어  
    -정상메일의 2%에서 '대출 단어  
    ---------- 주어진 데이터 -----------  
    문제) 방금 메일을 받음.  
    '대출'이라는 단어가 제목에서 발견됨.  
    스팸일 확률은?  
    P(스팸|대출)=P(대출|스팸)P(스팸) / P(대출)  
    =0.95*0.8 / P(대출)  
    P(대출)=P(대출|스팸)P(스팸) + P(대출|정상)P(정상)  
          = 0.95 * 0.8 + 0.02 * 0.2 = 0.764   
    P(스팸|대출) = 0.95 * 0.8 / 0.764 = ???

### 1. 스팸 메일 (나이브 베이지안 필터기)
    - P(A), P(B), P(B|A), P(A|B)
    - P(A|B) = P(B|A)P(A) / P(B)
    - P(정상|텍스트), P(비정상|텍스트)
    - P(정상|텍스트) = (P(정상|텍스트)*P(정상메일)) / P(텍스트)
    - P(스팸|텍스트) = (P(텍스트|스팸)*P(스팸)) / P(텍스트)
    => 단어가 3개(단어는 서로 독립)
    - 나이브 베이즈 분류기
        P(정상|텍스트) = P(w1|정상메일)*P(w2|정상메일)*P(w3|정상메일)=P(w1,w2,w3|정상메일)
        P(스팸|텍스트) = P(w1|스팸)*P(w2|스팸)*P(w3|스팸)
        단어 순서 고려 x
    - 만약 텍스트 있는 단어 중 어떤 단어가 받은 메일에 없으면 전체 확률 값이 0이 되므로 이를 방지하기 위해 +1을 해주는데 이를, '라플라스 스무딩'이라고 함 

### Bag of Words(BOW) : 단어들의 등장 횟수로 표현(단어가방)
    1) 주어진 단어에 대해 고유의 인덱스 부여
    2) 단어의 등장 횟수 벡터 생성

In [1]:
from konlpy.tag import Okt
okt=Okt()

C:\Users\user\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [22]:
import re
token="오늘은 금요일입니다. 내일은 토요일입니다. 다음주 화요일에는 특강이 있습니다"
# 정규표현식을 사용하여 token에 지정되 문자 중에서 (.)을 제거하세요.
token=re.sub('[.]','',token)
token

'오늘은 금요일입니다 내일은 토요일입니다 다음주 화요일에는 특강이 있습니다'

In [36]:
token=okt.morphs(token)
token

['오늘',
 '은',
 '금요일',
 '입니다',
 '내일',
 '은',
 '토요일',
 '입니다',
 '다음주',
 '화요일',
 '에는',
 '특강',
 '이',
 '있습니다']

In [15]:
# 1) 주어진 단어에 대해 고유의 인덱스 부여(0~11)
# 2) 단어의 등장 횟수 벡터 생성
# 1) => {'오늘':0,....'있습니다':11}
# 2) => [1,2,1,1,1,2,1,1,1,....]
token_dic={}
for w in token_list:
    if w not in token_dic:
        token_dic[w] = 1
    else :
        token_dic[w] += 1
print(token_dic)

{'오늘': 1, '은': 2, '금요일': 1, '입니다': 2, '내일': 1, '토요일': 1, '다음주': 1, '화요일': 1, '에는': 1, '특강': 1, '이': 1, '있습니다': 1}


In [38]:
word2index={}
bow=[]
for voc in token:
    if voc not in word2index.keys():
        word2index[voc]=len(word2index)
        bow.insert(len(word2index)-1,1)
    else:
        index=word2index.get(voc)
        bow[index]=bow[index]+1
print(word2index)

{'오늘': 0, '은': 1, '금요일': 2, '입니다': 3, '내일': 4, '토요일': 5, '다음주': 6, '화요일': 7, '에는': 8, '특강': 9, '이': 10, '있습니다': 11}


In [21]:
bow

[1, 1, 2, 7, 1, 3, 4, 2, 3, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
text=['you know I want your love. because I love you.']
# vec=CountVectorizer() #BOW 생성 클래스
# vec=CountVectorizer(stop_words=['I']) # 불용어 사용(사용자 정의)
vec=CountVectorizer(stop_words='english') #불용어 사용(사용자 정의)
print(vec.fit_transform(text).toarray()) #각 단어별 개수를 세며, 변환
print(vec.vocabulary_)

[[1 2 1]]
{'know': 0, 'want': 2, 'love': 1}


### 2. 문장간 유사도 조사(N-gram)
    (1, target) 
    오늘 역삼에서 맛있는 돈가스를 먹었다.
    (2, source) 
    역삼에서 먹었던 오늘의 돈가스는 맛있었다.
    (2)번 원문장에 대해 (1)번 대상문장이 어느정도 유사한지 출력?
    (1)번 문장에 대해 n=2로 하여 문장 분리하면
    => '오늘','늘','역','역삼',...,'다.'
    => 만약 길이가 20이라 가정하고, 
    (2)번 원문장에 대해서도 n=2로 하여 문장 분리했을 때, 
    공통으로 존재하는 단어가 5개 존재한다면, 
    5/20 = 25% 유사도를 갖는다. 
    
    두 문장의 유사도? (using n-gram)
    ex) n=2
    (1) '오늘', '늘', '역', '역삼',...'다.' => 20개 중 5개가 (2)과 일치
    (2) '역삼','삼에','에서',....'다. 

In [103]:
target="오늘 역삼에서 맛있는 돈가스를 먹었다."
source="역삼에서 먹었던 오늘의 돈가스는 맛있었다."

In [129]:
def ngram(s,num):
    res=[]
    slen=len(s)-num+1
    for i in range(slen):
        ss=s[i:i+num]
        res.append(ss)
    return res

def diff_ngram(sa, sb, num):
    a = ngram(sa, num)
    b = ngram(sb, num)
    r=[]
    cnt=0
    for i in a:
        for j in b:
            if i == j:
                cnt += 1
                r.append(i)
    return cnt/len(a), r

r2, word2 = diff_ngram(target,source,2)
r3, word3 = diff_ngram(target,source,3)
print('2-gram:', r2, word2)
print('3-gram:', r3, word3)

2-gram: 0.75 ['오늘', '역삼', '삼에', '에서', '서 ', ' 맛', '맛있', '는 ', ' 돈', '돈가', '가스', ' 먹', '먹었', '었다', '다.']
3-gram: 0.42105263157894735 ['역삼에', '삼에서', '에서 ', ' 맛있', ' 돈가', '돈가스', ' 먹었', '었다.']


In [130]:
from sklearn.feature_extraction.text import CountVectorizer
corpus=['you know I want your love',
       'I like you',
       'what should i do']

In [131]:
# DTM 생성
vec=CountVectorizer()
print(vec.fit_transform(corpus).toarray())
print(vec.vocabulary_)

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [132]:
# tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfv=TfidfVectorizer().fit(corpus)
print("="*50)
print(tfidfv.fit_transform(corpus).toarray())
print("="*50)
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


### 나이브 베이즈 분류기

In [133]:
from sklearn.datasets import fetch_20newsgroups
# 2만여편의 뉴스데이터
newsdata=fetch_20newsgroups(subset="train")
print(newsdata.keys())
print(len(newsdata.data), len(newsdata.filenames), len(newsdata.target_names))
print(newsdata.target_names) # 20개의 뉴스 그룹 확인
print(newsdata.target[0]) #0번 뉴스가 7번 카테고리에 해당한다는 의미(rec.autos)
print(newsdata.target_names[7]) # 20개의 그룹 중 7번째

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])
11314 11314 20
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
7
rec.autos


In [134]:
print(newsdata.data[1]) # 뉴스 데이터의 1번째 뉴스 
print(newsdata.target[1]) #4
print(newsdata.target_names[4]) #comp.sys.mac.hardware : 하드웨어에 속하는 글

From: guykuo@carson.u.washington.edu (Guy Kuo)
Subject: SI Clock Poll - Final Call
Summary: Final call for SI clock reports
Keywords: SI,acceleration,clock,upgrade
Article-I.D.: shelley.1qvfo9INNc3s
Organization: University of Washington
Lines: 11
NNTP-Posting-Host: carson.u.washington.edu

A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered this
poll. Thanks.

Guy Kuo <guykuo@u.washington.edu>

4
comp.sys.mac.hardware


In [135]:
testdata=fetch_20newsgroups(subset="test")
print(testdata.keys())
print(len(testdata.data)) #7532

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])
7532


In [136]:
alldata=fetch_20newsgroups(subset="test")
print(alldata.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [137]:
# 1. 텍스트 -> bow 생성 (bow에 등장하지 않은 단어는 OOV)
# 테스트 단어를 분류기 넣었을 때 만약 BOW에 없는 단어인 경우 라플라스 스무딩에 의해 +1이 됨
# 보통 분류기 생성 전, dtm을 tf-idf 행렬로 변환줌(성능이 훨씬 올라감)
dtmVector=CountVectorizer()
xtrainDtm=dtmVector.fit_transform(newsdata.data)
xtrainDtm.shape

(11314, 130107)

In [138]:
# 2. dtm -> tf-idf 행렬 변환
from sklearn.feature_extraction.text import TfidfTransformer # dtm을 ti-idf 행렬로 변환해주는 클래스
tfidf_transformer=TfidfTransformer()
tfidfv=tfidf_transformer.fit_transform(xtrainDtm)

In [139]:
# 3. 나이브베이즈 분류기 모델 생성
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(tfidfv, newsdata.target) # alpha:라플라스 스무딩 값, train 데이터와 labels

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [140]:
testdata=fetch_20newsgroups(subset='test',shuffle=True)
xtestDtm=dtmVector.transform(testdata.data)
tfidf_test=tfidf_transformer.transform(xtestDtm)

In [141]:
from sklearn.metrics import accuracy_score
predicted=model.predict(tfidf_test)
print("정확도:", accuracy_score(testdata.target, predicted))

정확도: 0.7738980350504514
